<a href="https://colab.research.google.com/github/jungeun919/Pytorch_study/blob/main/Pytorch_Tutorial/Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 모델 매개변수 최적화

In [1]:
# 전제조건 코드 (Datasets&DataLoaders, Build_Model의 코드)
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw
Processing...


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:479: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Done!


# 하이퍼파라미터 (Hyperparameters)

- 모델 최적화 프로세스를 제어할 수 있는 조정 가능한 매개변수

In [2]:
learning_rate= 1e-3
batch_size = 64
epochs = 5

# 손실함수

- nn.MSELoss: 평균제곱오차, 회귀 작업
- nn.NLLLoss: 음수 로그 가능성, 분류
- nn.CrossEntropyLoss: nn.LogSoftmax와 nn.NLLLoss의 결합

In [3]:
loss_fn = nn.CrossEntropyLoss()

# Optimizer

최적화는 다음 세단계로 이루어짐
1. optimizer.zero_grad()
2. loss.backward()
3. optimizer.step()

In [4]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# Full Implementation

In [5]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (x, y) in enumerate(dataloader):
        pred = model(x)
        loss = loss_fn(pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(x)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for x, y in dataloader:
            pred = model(x)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [6]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n--------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done")

Epoch 1
--------------------
loss: 2.302755 [    0/60000]
loss: 2.298337 [ 6400/60000]
loss: 2.295365 [12800/60000]
loss: 2.293030 [19200/60000]
loss: 2.287457 [25600/60000]
loss: 2.284572 [32000/60000]
loss: 2.292176 [38400/60000]
loss: 2.290732 [44800/60000]
loss: 2.278401 [51200/60000]
loss: 2.264705 [57600/60000]
Test Error: 
 Accuracy: 25.9%, Avg loss: 0.035553 

Epoch 2
--------------------
loss: 2.273783 [    0/60000]
loss: 2.258538 [ 6400/60000]
loss: 2.253002 [12800/60000]
loss: 2.254544 [19200/60000]
loss: 2.239861 [25600/60000]
loss: 2.230038 [32000/60000]
loss: 2.247470 [38400/60000]
loss: 2.240062 [44800/60000]
loss: 2.202432 [51200/60000]
loss: 2.189651 [57600/60000]
Test Error: 
 Accuracy: 33.6%, Avg loss: 0.034448 

Epoch 3
--------------------
loss: 2.199291 [    0/60000]
loss: 2.175422 [ 6400/60000]
loss: 2.162940 [12800/60000]
loss: 2.177373 [19200/60000]
loss: 2.167099 [25600/60000]
loss: 2.121350 [32000/60000]
loss: 2.150049 [38400/60000]
loss: 2.121041 [44800/6000